In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [22]:
def grad_U(Ui, Yij, Vj, ai, bj, reg, eta):
    """
    Takes as input Ui (the ith row of U), a training point Yij, the column
    vector Vj (jth column of V^T), reg (the regularization parameter lambda),
    and eta (the learning rate).

    Returns the gradient of the regularized loss function with
    respect to Ui multiplied by eta.
    """
    reg_grad = Ui * reg - (Yij - (np.dot(Ui, Vj) + ai + bj)) * Vj
    return eta * reg_grad

def grad_V(Vj, Yij, Ui, ai, bj, reg, eta):
    """
    Takes as input the column vector Vj (jth column of V^T), a training point Yij,
    Ui (the ith row of U), reg (the regularization parameter lambda),
    and eta (the learning rate).

    Returns the gradient of the regularized loss function with
    respect to Vj multiplied by eta.
    """
    reg_grad = Vj * reg - (Yij - (np.dot(Vj, Ui) + ai + bj)) * Ui
    return eta * reg_grad

def grad_a(ai, Yij, Ui, Vj, bj, eta):
    """
    Takes as input the bias term ai, a training point Yij, Ui (the ith row of U),
    column vector Vj (jth column of V^T), the bias term bj, and eta (the learning rate).

    Returns the gradient of the regularized loss function with
    respect to ai multiplied by eta.
    """
    return eta * -(Yij - (np.dot(Vj, Ui) + ai + bj))

def grad_b(bj, Yij, Ui, Vj, ai, eta):
    """
    Takes as input the bias term bj, a training point Yij, Ui (the ith row of U),
    column vector Vj (jth column of V^T), the bias term ai, and eta (the learning rate).

    Returns the gradient of the regularized loss function with
    respect to bj multiplied by eta.
    """
    return eta * -(Yij - (np.dot(Vj, Ui) + ai + bj))


def get_err(U, V, a, b, Y, reg=0.0):
    """
    Takes as input a matrix Y of triples (i, j, Y_ij) where i is the index of a user,
    j is the index of a movie, and Y_ij is user i's rating of movie j and
    user/movie matrices U and V.

    Returns the mean regularized squared-error of predictions made by
    estimating Y_{ij} as the dot product of the ith row of U and the jth column of V^T.
    """
    reg_term = reg * (np.linalg.norm(U, 'fro')**2 + np.linalg.norm(V, 'fro')**2) / 2
    
    least_square_sum = 0
    
    for (i, j, Yij) in Y:
        Ui = U[i - 1]
        Vj = V[j - 1]
        ai = a[i - 1]
        bj = b[j - 1]
        least_square_sum += (Yij - (np.dot(Ui, Vj) + ai + bj))**2
    
    least_square_sum /= 2
    
    return (reg_term + least_square_sum) / Y.shape[0]


def train_model(M, N, K, eta, reg, Y, eps=0.0001, max_epochs=300):
    """
    Given a training data matrix Y containing rows (i, j, Y_ij)
    where Y_ij is user i's rating on movie j, learns an
    M x K matrix U and N x K matrix V such that rating Y_ij is approximated
    by (UV^T)_ij. Also learns bias terms a and b for users and movies.

    Uses a learning rate of <eta> and regularization of <reg>. Stops after
    <max_epochs> epochs, or once the magnitude of the decrease in regularized
    MSE between epochs is smaller than a fraction <eps> of the decrease in
    MSE after the first epoch.

    Returns a tuple (U, V, a, b, err) consisting of U, V, a, b and the
    unregularized MSE of the model.
    """
    U = np.random.uniform(-0.5, 0.5, size=(M, K))
    V = np.random.uniform(-0.5, 0.5, size=(N, K))
    a = np.random.uniform(-0.5, 0.5, size=(M))
    b = np.random.uniform(-0.5, 0.5, size=(N))
    initial_delta = 0
    old_loss = get_err(U, V, a, b, Y, reg)
    
    for epoch in range(max_epochs):
        
        indices = np.random.permutation(len(Y))
        
        for index in indices:
            (i, j, Yij) = Y[index]
            Ui = U[i - 1]
            Vj = V[j - 1]
            ai = a[i - 1]
            bj = b[j - 1]
            new_Ui = Ui - grad_U(Ui, Yij, Vj, ai, bj, reg, eta)
            new_Vj = Vj - grad_V(Vj, Yij, Ui, ai, bj, reg, eta)
            new_ai = ai - grad_a(ai, Yij, Ui, Vj, bj, eta)
            new_bj = bj - grad_b(bj, Yij, Ui, Vj, ai, eta)
            U[i - 1] = new_Ui
            V[j - 1] = new_Vj
            a[i - 1] = new_ai
            b[j - 1] = new_bj
        
        new_loss = get_err(U, V, a, b, Y, reg)
            
        if epoch == 0:
            initial_delta = new_loss - old_loss
        else:
            delta = new_loss - old_loss
            if abs(delta / initial_delta) <= eps:
                break
        
        old_loss = new_loss
        
    err = get_err(U, V, a, b, Y)
    return (U, V, a, b, err)

In [23]:
Y_train = np.loadtxt('./data/train.txt').astype(int)
Y_test = np.loadtxt('./data/test.txt').astype(int)

# data is 1 indexed.

M = max(max(Y_train[:,0]), max(Y_test[:,0])).astype(int) # users
N = max(max(Y_train[:,1]), max(Y_test[:,1])).astype(int) # movies
print("Factorizing with ", M, " users, ", N, " movies.")

reg = 0.1
eta = 0.03 # learning rate
K = 20

# Use to compute Ein and Eout
U, V, a, b, err = train_model(M, N, K, eta, reg, Y_train)
E_in = err
E_out = get_err(U, V, a, b, Y_test)
print(E_in, E_out)


Factorizing with  943  users,  1682  movies.
0.2619575857470955 0.4158797841374228
